In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def uk_IO_widget(shockSector_init = ['Financial and insurance'], shockSign_init = 'Negative',
                 shockSize_init = 10, NumRounds_init = 5, Normalise_init = False, 
                 plotType_init = 'Shocks over time'):
    
    shockSector_list = widgets.SelectMultiple(options=['Agriculture',
                                        'Production',
                                        'Construction',
                                        'Distribution, transport, hotels and restaurants',
                                        'Information and communication',
                                        'Financial and insurance',
                                        'Real estate',
                                        'Professional and support activities',
                                        'Government, health & education',
                                        'Other services'],
                                value = shockSector_init,
                                disabled=False,
                                layout={'width': 'max-content'})
    shockSign_list = widgets.Dropdown(options=['Positive', 'Negative'],
                                value = shockSign_init,
                                description='Sign:',
                                disabled=False)
    plotType_list = widgets.Dropdown(options=['Shocks over time', 'Cumulative shocks'],
                                value = plotType_init,
                                description='Diagram:',
                                disabled=False)
    shockSize_slider = widgets.IntSlider(min=1,
                                         max=80,
                                         step=1,
                                         description= r'% size :',
                                         value = shockSize_init,
                                         disabled=False,
                                         continuous_update=False)
    NumRounds_slider = widgets.IntSlider(min=1,
                                         max=20,
                                         step=1,
                                         description= r'N° of rounds:',
                                         value = NumRounds_init,
                                         disabled=False,
                                         continuous_update=False)
    Normalise_check = widgets.Checkbox(value = Normalise_init,
                                   description='Display shocks in %',
                                   disabled=False,
                                   indent=True)
    
    def uk_IO_plot(shockSector, shockSign, shockSize, NumRounds, Normalise, 
                   plotType):

        sectorDict = {'Agriculture' : 0,
                    'Production' : 1,
                    'Construction' : 2,
                    'Distribution, transport, hotels and restaurants' : 3,
                    'Information and communication' : 4,
                    'Financial and insurance' : 5,
                    'Real estate' : 6,
                    'Professional and support activities' : 7,
                    'Government, health & education' : 8,
                    'Other services' : 9}

        colors = {'Agriculture' :'b',
                  'Production' : 'k',
                  'Construction' : 'r',
                  'Distribution, transport, hotels and restaurants' : 'g',
                  'Information and communication' : 'c',
                  'Financial and insurance' : 'b--',
                  'Real estate' : 'k--',
                  'Professional and support activities' : 'r--',
                  'Government, health & education' : 'g--',
                  'Other services' : 'c--'}

        signs = {'Positive': 1,
                 'Negative': -1}

        # Hardcoded UK Input/Output data to avoid data dependencies
        IO = [
            [6246,  17458,     0,   618,     0,     2,    65,   0,      80,    63],
            [8409, 367066, 57463, 99070, 16041,  8583,  5463, 26135, 75138, 14615],
            [ 353,   7816, 96507,  5117,   177,  4787, 18953,  3527,  7223,   349],
            [1389,  23737,  1339, 76591,  5697, 17617,   684, 18503, 15250,  2254],
            [ 465,   8272,  2641, 11955, 34882, 20785,  1893, 11425, 11123,  4190],
            [1395,  18151,  4514, 13071,  2866, 39145, 27991,  9098, 10320,  1485],
            [   0,   1875,   395, 17849,  3583,  4050,  1914,  5850,  8587,  1489],
            [1837,  41941, 16618, 55125, 24720, 46022,  8383, 93386, 38383, 14367],
            [  27,   5325,  1189,  6479,   678,  2539,  3124,  5885, 32118,   280],
            [  71,    310,     1,   196,  2385,  1617,    29,   773,  5277,  7897]]

        X = np.asmatrix([[55226],
                         [1599529],
                         [324055],
                         [389993],
                         [247981],
                         [288304],
                         [355744],
                         [519509],
                         [532486],
                         [121469]])

        D = np.asmatrix([[30694],
                        [921546],
                        [179246],
                        [226932],
                        [140350],
                        [160268],
                        [310152],
                        [178727],
                        [474842],
                        [102913]])

        IOmat = np.asmatrix(IO)
        TCmat = IOmat / np.transpose(X)
        I = np.identity(10)
        TCinv = np.linalg.inv(I-TCmat)

        # Apply shocks to sectors
        dD = np.zeros([10,1])
        for sec_choice in shockSector:
            sector = sectorDict[sec_choice]
            dD[sector] = signs[shockSign]*(shockSize/100)*D[sector]

        shockRounds = np.zeros([10,NumRounds])
        dXround = dD
        for shockRound in range(NumRounds):
            dXround = TCmat*dXround
            shockRounds[:,shockRound] = dXround.flatten()

        # Concatenate the initial shock, calculate cumulative effect and 
        # the % captured by the rounds
        shockRoundsCum = np.cumsum(
                np.concatenate((dD,shockRounds), axis = 1), axis = 1)
        shockRounds = np.concatenate((dD,shockRounds),axis = 1)
        dX = TCinv*dD
        cover = float( 100*sum(shockRoundsCum[:,-1])/sum(dX))

        # Normalise if required, get bounds
        if Normalise is True:
            shockRounds /=  X    

        yMax = 1.2*max(shockRounds.flatten())        
        yMin = 1.2*min(shockRounds.flatten())
        yMaxCum = 1.1*sum(shockRoundsCum[:,-1]) 

        # Create figure
        fig, ax = plt.subplots(figsize=(20,10))
        roundsVec = np.arange(0,NumRounds+1)

        # Plot shocks by sector and round
        if plotType == 'Shocks over time':

            for sector in sectorDict:
                ax.plot(roundsVec, shockRounds[sectorDict[sector],:], 
                        colors[sector], linewidth=2, alpha=0.6, label=sector)

            # Add legend and format axes to look nice
            ax.autoscale(enable=True, axis='both', tight=True)
            if shockSign == 'Positive':
                ax.set_ylim(top = yMax, bottom = 0)
                loc = 'upper right'
                ax.tick_params(labelsize=20)
            elif shockSign == 'Negative':
                ax.set_ylim(top = 0, bottom = yMin)
                loc = 'lower right'
                ax.tick_params(labelsize=20,labelbottom=False,labeltop=True)

            ax.set_xlim(right = NumRounds, left = 0)
            ax.legend(loc=loc, frameon=False,prop={'size':20})
            ax.spines['bottom'].set_position('zero')
            ax.spines['top'].set_visible(False)
            ax.spines['left'].set_position('zero')
            ax.spines['right'].set_visible(False)
            ax.set_xlabel('Rounds', fontdict = {'fontsize': 25},position=(1, 0))
            ax.set_ylabel('Impact', fontdict = {'fontsize': 25},position=(0, 1),
                          rotation=0)
            ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
            ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)


        elif plotType == 'Cumulative shocks':

            bottomVals = np.zeros(NumRounds+1)

            for sector in sectorDict:

                barColor = colors[sector][0]
                if '--' in colors[sector]:
                    if sectorDict[sector] % 2 == 0:
                        barHatch = '/'
                    else:
                        barHatch = '\\'
                else:
                    barHatch = ''

                ax.bar(roundsVec, shockRoundsCum[sectorDict[sector],:], 
                       width = 0.5, bottom = bottomVals, color = barColor, 
                       hatch = barHatch, alpha = 0.6, label=sector)
                bottomVals += shockRoundsCum[sectorDict[sector],:]

            coverStr = '{:d} rounds capture {:.2f}% of the total effect'.format(
                NumRounds,cover)

            ax.autoscale(enable=True, axis='both', tight=True)
            if shockSign == 'Positive':
                ax.set_ylim(top = yMaxCum, bottom = 0)
                legLoc = 'upper center'
                annotLoc1 = 1
                annotLoc2 = 0.90
                ax.tick_params(labelsize=20)
            elif shockSign == 'Negative':
                ax.set_ylim(top = 0, bottom = yMaxCum)
                legLoc = 'lower center'
                annotLoc1 = 0
                annotLoc2 = 0.05                
                ax.tick_params(labelsize=20,labelbottom=False,labeltop=True)

            ax.annotate(coverStr,[0,annotLoc1], xytext = [0.05,annotLoc2], 
                        xycoords ='axes fraction', fontsize = 25, clip_on = True)
                
            ax.set_xlim(right = NumRounds+0.5, left = -0.5)
            ax.legend(loc=legLoc, frameon=False,prop={'size':20},ncol=3, 
                      bbox_to_anchor=(0.5, -0.25))
            ax.spines['bottom'].set_position('zero')
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.set_xlabel('Rounds', fontdict = {'fontsize': 25},
                          position=(1.05, 0.05))
            ax.set_ylabel('Cumulative Impact', fontdict = {'fontsize': 25},
                          position=(0, 1), rotation=0)
            ax.tick_params(labelsize=20,labelbottom=False,labeltop=True)

        plt.tight_layout()
    
    out = widgets.interactive_output(uk_IO_plot, {'shockSector': shockSector_list,
                                                   'shockSign': shockSign_list,
                                                   'shockSize': shockSize_slider, 
                                                   'NumRounds': NumRounds_slider,
                                                   'plotType': plotType_list,                                                      
                                                   'Normalise': Normalise_check})
    
    lblStr = 'Pick sector(s) to shock:'
    output = widgets.VBox([out,
                  widgets.HBox([plotType_list,
                                NumRounds_slider,
                                Normalise_check]),
                  widgets.HBox([widgets.Label(value=lblStr)]),      
                  widgets.HBox([shockSector_list,
                                shockSign_list,
                                shockSize_slider])
                          ])
    
    display(output)


In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Input-Output analysis: an application of matrices in economics

The standard economics textbooks that you will be using this year will talk about the fact there are many markets for various goods and services. You will hear about the fact that each market has a set of suppliers and a set of consumers but most of the time  we won't mention how these markets are connected. in real life if you can easily imagine that it is possible for an individual or a firm to be a supplier in one market and a consumer in the other. similarly a given good could be Used as an input into the production of another good or just bought by a final consumer. A good example of this are car tyres. They can be bought by car factories to put on brand new cars, or they can be bought by garages as spare parts or they can be bought by individuals who want to change a tire themselves.  Computers are another good example,  bought by firms and final consumers alike.

As we will see below, the fact that firms have to purchase inputs from other firms in order to be able to produce their own output is one of the things that can explain how a shock in one sector of the economy can propagate to other sectors of the economy. Basically, the structure of economic production  is important in itself! This structure can be represented and analysed with matrices. Again, like previous notebooks, don't be worried if this looks complicated, the purpose of this material is to illustrate why the math is important.

#### The UK input-output data

Let's start by looking at how that data is summarised in an input-output table (sometimes also called supply-use tables).  The table below shows the United Kingdom's input output table for 10 economic sectors in the year  2017. Note that these tables can end up being very very detailed with hundreds of different sectors.  While these tables are very useful forgetting a detailed picture of the economy they are way too big  for us to use here. So 10 sectors is going to be good enough for us.

|                                                   | <div style="padding: 5px ;   background-color: salmon">Agriculture</div> | <div style="padding: 5px ;   background-color: salmon">Production</div> | <div style="padding: 5px ;   background-color: salmon">Construction</div> | <div style="padding: 5px ;   background-color: salmon">Distribution, transport, hotels and   restaurants</div> | <div style="padding: 5px ;   background-color: salmon">Information and communication</div> | <div style="padding: 5px ;   background-color: salmon">Financial and insurance</div> | <div style="padding: 5px ;   background-color: salmon">Real estate</div> | <div style="padding: 5px ;   background-color: salmon">Professional and support   activities</div> | <div style="padding: 5px ;   background-color: salmon">Government, health &   education</div> | <div style="padding: 5px ;   background-color: salmon">Other services</div> | \| | <div style="padding: 5px ;   background-color: skyblue">Consumption</div> | <div style="padding: 5px ;   background-color: skyblue">Investment</div> | <div style="padding: 5px ;   background-color: skyblue">Exports of goods to EU</div> | <div style="padding: 5px ;   background-color: skyblue">Exports of goods to Rest of   World</div> | <div style="padding: 5px ;   background-color: skyblue">Exports of services</div> | \| | <div style="padding: 5px ;   background-color: salmon">Intermediate consumption</div> | <div style="padding: 5px ;   background-color: skyblue">Final Demand</div> | Production |
|---------------------------------------------------|--------------------------------------------------------------------------|-------------------------------------------------------------------------|---------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------|--------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------|----|---------------------------------------------------------------------------|--------------------------------------------------------------------------|--------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------|----|---------------------------------------------------------------------------------------|----------------------------------------------------------------------------|------------|
| Agriculture                                       | 6246                                                                     | 17458                                                                   | 0                                                                         | 618                                                                                                            | 0                                                                                          | 2                                                                                    | 65                                                                       | 0                                                                                                  | 80                                                                                            | 63                                                                          | \| | 25993                                                                     | 1373                                                                     | 2190                                                                                 | 888                                                                                               | 250                                                                               | \| | 24532                                                                                 | 30694                                                                      | 55226      |
| Production                                        | 8409                                                                     | 367066                                                                  | 57463                                                                     | 99070                                                                                                          | 16041                                                                                      | 8583                                                                                 | 5463                                                                     | 26135                                                                                              | 75138                                                                                         | 14615                                                                       | \| | 491127                                                                    | 90553                                                                    | 159693                                                                               | 165747                                                                                            | 14426                                                                             | \| | 677983                                                                                | 921546                                                                     | 1599529    |
| Construction                                      | 353                                                                      | 7816                                                                    | 96507                                                                     | 5117                                                                                                           | 177                                                                                        | 4787                                                                                 | 18953                                                                    | 3527                                                                                               | 7223                                                                                          | 349                                                                         | \| | 2198                                                                      | 175062                                                                   | 0                                                                                    | 0                                                                                                 | 1986                                                                              | \| | 144809                                                                                | 179246                                                                     | 324055     |
| Distribution, transport, hotels and   restaurants | 1389                                                                     | 23737                                                                   | 1339                                                                      | 76591                                                                                                          | 5697                                                                                       | 17617                                                                                | 684                                                                      | 18503                                                                                              | 15250                                                                                         | 2254                                                                        | \| | 185136                                                                    | 0                                                                        | 0                                                                                    | 0                                                                                                 | 41796                                                                             | \| | 163061                                                                                | 226932                                                                     | 389993     |
| Information and communication                     | 465                                                                      | 8272                                                                    | 2641                                                                      | 11955                                                                                                          | 34882                                                                                      | 20785                                                                                | 1893                                                                     | 11425                                                                                              | 11123                                                                                         | 4190                                                                        | \| | 55238                                                                     | 43837                                                                    | 1813                                                                                 | 1816                                                                                              | 37646                                                                             | \| | 107631                                                                                | 140350                                                                     | 247981     |
| Financial and insurance                           | 1395                                                                     | 18151                                                                   | 4514                                                                      | 13071                                                                                                          | 2866                                                                                       | 39145                                                                                | 27991                                                                    | 9098                                                                                               | 10320                                                                                         | 1485                                                                        | \| | 75827                                                                     | 0                                                                        | 0                                                                                    | 0                                                                                                 | 84441                                                                             | \| | 128036                                                                                | 160268                                                                     | 288304     |
| Real estate                                       | 0                                                                        | 1875                                                                    | 395                                                                       | 17849                                                                                                          | 3583                                                                                       | 4050                                                                                 | 1914                                                                     | 5850                                                                                               | 8587                                                                                          | 1489                                                                        | \| | 297693                                                                    | 10861                                                                    | 0                                                                                    | 0                                                                                                 | 1598                                                                              | \| | 45592                                                                                 | 310152                                                                     | 355744     |
| Professional and support activities               | 1837                                                                     | 41941                                                                   | 16618                                                                     | 55125                                                                                                          | 24720                                                                                      | 46022                                                                                | 8383                                                                     | 93386                                                                                              | 38383                                                                                         | 14367                                                                       | \| | 42189                                                                     | 40735                                                                    | 25                                                                                   | 19                                                                                                | 95759                                                                             | \| | 340782                                                                                | 178727                                                                     | 519509     |
| Government, health & education                    | 27                                                                       | 5325                                                                    | 1189                                                                      | 6479                                                                                                           | 678                                                                                        | 2539                                                                                 | 3124                                                                     | 5885                                                                                               | 32118                                                                                         | 280                                                                         | \| | 462111                                                                    | 1007                                                                     | 0                                                                                    | 0                                                                                                 | 11724                                                                             | \| | 57644                                                                                 | 474842                                                                     | 532486     |
| Other services                                    | 71                                                                       | 310                                                                     | 1                                                                         | 196                                                                                                            | 2385                                                                                       | 1617                                                                                 | 29                                                                       | 773                                                                                                | 5277                                                                                          | 7897                                                                        | \| | 96020                                                                     | -376                                                                     | 139                                                                                  | 5137                                                                                              | 1993                                                                              | \| | 18556                                                                                 | 102913                                                                     | 121469     |

Data for 2017, all values in millions of £.
Source: [Input–output supply and use tables - summary tables, Office of National Statistics](https://www.ons.gov.uk/economy/nationalaccounts/supplyandusetables/datasets/inputoutputsupplyandusetablessummarytables)

Let's explain how this table works. The rows show the amount  of output consumed for each of the 10 sectors. The columns show the amount consumed by the type of consumer. So for example £618 million worth of agricultural output is consumed by the distribution sector. The table itself can be divided into 3 parts. On the very left, where the headers are in red, you have the intermediate consumption matrix which tells us how much of the input of one sector is used to produce the output of another sector. Note that because any sector can consume from any other sector this is essentially a 10 by 10 matrix! We will look at this in more detail in just a bit. 

The second part of the table, to the right with headers in blue, are the sources of final consumption. These are forms of consumption that do not result in the production of another good, or at least not in the UK (some of the exports might of course be used as inputs in other countries, but because that doesn't count as UK production we don't care about that).  The main sources of final consumption are consumption, where the good or service is just used by household, investment, where the good or service is added to the capital stock of the country and finally exports. Note that the table separates exports to the EU and to the rest of the world, which will be useful for running a Brexit impact assessment. 

At the very right of the table we have some totals. The first column (red header) is the sum of the intermediate consumption matrix over its columns (i.e. it gives the total amount of agricultural production etc used up as part of the production process). The second column, with a blue header, is the sum of all the final demands for each sector and the final column is the amount of output produced by each sector. 


#### Packaging the data into matrices and vectors

As explained in the lecture, you can always think of a table of data as matrix, and this table is no exception. This will allow us to write the system of equations that summarises the UK economic production for these sectors. In order to  do this we first need to slightly modify the matrix of intermediate consumption (on the left, with red headers).  What we do is divide each column by the level of output of the corresponding sector. So that means we divide the first column by the total output of the agricultural sector, the second column by the total output of the production sector (note that production means industry/manufacturing in this case), the third column by the total output of the construction sector etc. This produces is a matrix called the **matrix of technical coefficients**, shown below, which we will label $\color{red}{A}$. 

$$\color{red}{A}=\left( {\begin{array}{cccccccccc}
0.11310 & 0.01091 & 0.00000 & 0.00158 & 0.00000 & 0.00001 & 0.00018 & 0.00000 & 0.00015 & 0.00052 \\
0.15227 & 0.22948 & 0.17732 & 0.25403 & 0.06469 & 0.02977 & 0.01536 & 0.05031 & 0.14111 & 0.12032 \\
0.00639 & 0.00489 & 0.29781 & 0.01312 & 0.00071 & 0.01660 & 0.05328 & 0.00679 & 0.01356 & 0.00287 \\
0.02515 & 0.01484 & 0.00413 & 0.19639 & 0.02297 & 0.06111 & 0.00192 & 0.03562 & 0.02864 & 0.01856 \\
0.00842 & 0.00517 & 0.00815 & 0.03065 & 0.14066 & 0.07209 & 0.00532 & 0.02199 & 0.02089 & 0.03449 \\
0.02526 & 0.01135 & 0.01393 & 0.03352 & 0.01156 & 0.13578 & 0.07868 & 0.01751 & 0.01938 & 0.01223 \\
0.00000 & 0.00117 & 0.00122 & 0.04577 & 0.01445 & 0.01405 & 0.00538 & 0.01126 & 0.01613 & 0.01226 \\
0.03326 & 0.02622 & 0.05128 & 0.14135 & 0.09969 & 0.15963 & 0.02356 & 0.17976 & 0.07208 & 0.11828 \\
0.00049 & 0.00333 & 0.00367 & 0.01661 & 0.00273 & 0.00881 & 0.00878 & 0.01133 & 0.06032 & 0.00231 \\
\end{array} } \right)$$

The size of it might be scary, but don't worry, the computer does all the work here. Before we look at how we use this, let's first try and understand what these coefficients mean by looking at the first column of $\color{red}{A}$, which corresponds to Agriculture. Remember that in the original table, the columns represent the intermediate consumption by the corresponding sector. Now that we have divided the first column by the total output of Agriculture, this tells us that to produce £1 of output in the agricultural sector we need to use £0.1131 worth of agricultural output, £0.1523 worth of production output,  £0.00639 worth of construction output etc. So you can think of this as the recipe or list of ingredients needed to produce £1 of agricultural output. Several things to note here:
- First  production in a sector often requires input from the same sector. For the example we use above, Agriculture, generating £1 of output requires spending about £0.15 on agricultural inputs. The example here might be a cattle farmer needing to feed his cattle over the winter, and buying feed produced on another farm.
- Second, this self consumption by sector can be relatively high. The best example of this is the production sector where in order to produce £1 of output, you need to spend £0.23 on inputs from the production sector itself! as you may suspect, this will be important when we look at how the economy responds to shocks.

The next thing we need to do do is package the columns of final demands and production (at the very right of the table) into two vectors: $\color{blue}{d}$ for final demand and $x$ for output:

$$\color{blue}{d}=\left( {\begin{array}{c}
30694 \\
921546 \\
179246 \\
226932 \\
140350 \\
160268 \\
310152 \\
178727 \\
474842 \\
102913 \\
\end{array} } \right)\qquad x = \left( {\begin{array}{c}
55226 \\
1599529 \\
324055 \\
389993 \\
247981 \\
288304 \\
355744 \\
519509 \\
532486 \\
121469 \\
\end{array} } \right)$$

We can now use all this write down the equation that we're going to use. If you look at the table row by row, production of each sector is equal to to the sum of final demand and intermediate consumption for that sector. What this means is simply that  every good or services produced by a sector is either sold as intermediate consumption or to final demand. This can be written down very simply as:

$$x = \color{red}{A}x+\color{blue}{d}$$

Note that for intermediate consumption we seem to be using $\color{red}{A}x$. First, multiplying the 10 by 10 matrix $\color{red}{A}$ and the 10 by 1 vector $x$ will produce a 10 by 1 vector, just like $x$ and $\color{blue}{d}$. This vectorwill be equal to the intermediate consumption column in the table because when building $A$ we divided each column by the corresponding output in $x$. That way, the matrix multiplication $\color{red}{A}x$ simply recovers the original table and sums over the columns to get a column vector.

This may seem a round-about way of doing things: why bother dividing the table data by $x$ to get $\color{red}{A}$ if we're going to undo this 2 seconds later?? Well, by using this trick we now have $x$ on both sides of the equation, so if we move $\color{red}{A}x$ to the left, we can factor it and use good old matrix inversion to solve for $x$! 

$$\begin{aligned}
x-\color{red}{A}x&=\color{blue}{d}\\
(I-\color{red}{A})x&=\color{blue}{d}\\
\end{aligned}$$

Remember that I is the identity matrix, which the matrix algebra equivalent of 1. To the value of $x$ given $\color{blue}{d}$, you guessed it, we need to invert the matrix $I-A$. Because you've carefully studied the lecture material, you know we need to check that the determinant is not zero. The computer can do that, and tells us $|I-A| \approx 0.196$. This is not zero, so we can invert the matrix, giving us:

$$x=(I-\color{red}{A})^{-1}\color{blue}{d}$$

Obviously, inverting a 10 by 10 matrix is not done by hand, the computer does this for you. The key thing is that we now have an equation that spits out the level of output per sector $x$ when we feed it the level of demand per sector $\color{blue}{d}$.

### Practical uses of the input-output formula

You may well ask: what's the point of this, because the table already gives us the level of output $x$, we don't need to calculate it!! True, but as you will see the formula also allows us to calculate how output ***changes*** when demand changes. For example you can put a shock on the level of demand in one sector of the economy, for example production, and see how this affects the production of *all* sectors of the economy: this can be done by adding a vector $\Delta d$ to the existing demand vector, where are all the entries are equal to 0 except for the production sector, whose demand is changed by an amount $a$ (positive or negative). 

$$\color{blue}{\Delta d}=\left( {\begin{array}{c}
0 \\
\color{blue}{a} \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
0 \\
\end{array} } \right)$$

Feeding this demand shock into our formula will tell us how the level of output for each sector is affected by that initial shock.

$$\Delta x=(I-\color{red}{A})^{-1}\color{blue}{\Delta d}$$

#### Tracing the ripple effect

Even better you can actually trace how the shock propagates through the economy. I won't go into too many technical details on how you do this but it has to do with the limit of a convergent series. If you have a number $0 < a < 1$, then the following geometric series converges to a fixed value. Try this on your calculator with $a$ = 0.5, and you will see that after adding the first few power terms you are already very close to 2.

$$1 + a + a^2 + a^3 +  a^4 + \dots = \frac{1}{1-a}$$

It turns out you can do the same with our matrix $\color{red}{A}$ as long as the determinant of the matrix $|\color{red}{A}|$is less than 1, which is always the case for input-output tables:

$$I + \color{red}{A} + \color{red}{A}^2 + \color{red}{A}^3 +  \color{red}{A}^4 + \dots = (I-\color{red}{A})^{-1}$$

Replacing this into the formula we got above shows that that you can decompose the total effect into a set of rounds which show how the initial shock ripples through the sectors of the economy. This is what the diagram below illustrates.

$$\Delta x=(I-\color{red}{A})^{-1}\color{blue}{\Delta d}  \quad = \quad \color{blue}{\Delta d} + \color{red}{A}\color{blue}{\Delta d} + \color{red}{A}^2\color{blue}{\Delta d} + \color{red}{A}^3\color{blue}{\Delta d} +  \color{red}{A}^4\color{blue}{\Delta d} + \dots$$


#### Payoff for getting this far: a crude impact analysis of Brexit

The original table at the top of the notebook breaks exports down between EU and non-EU exports. This allows us to see for example that the exports of the production sector to the EU (£160 billion) represents about 17% of final demand for that sector (£922 billion). What would happen, for example, if there is a no-deal Brexit, and exports of goods to the EU are reduced by about two-thirds? We know that final demand for the production sector would fall by about 12%, but what would be the effect on the output of other sectors? The diagrams below show the evolution of the shock round by round as well as cumulatively.


In [3]:
uk_IO_widget(shockSector_init = ['Production'], shockSign_init = 'Negative',
                 shockSize_init = 12, NumRounds_init = 5, Normalise_init = False, 
                 plotType_init = 'Shocks over time')

In [4]:
uk_IO_widget(shockSector_init = ['Production'], shockSign_init = 'Negative',
                 shockSize_init = 12, NumRounds_init = 5, Normalise_init = False, 
                 plotType_init = 'Cumulative shocks')

Both diagrams suggest that the shock remains confined mainly to the production sector, but grows over several rounds.  Only in later rounds to the negative effects spill out to other sectors This is because, as we pointed out above, the production sector is strongly connected to itself. Once the demand for final manufactured goods falls, the shock will propagate to all the upstream suppliers who produce parts as inputs.

Another example you might want to look at is the effect of a no-deal Brexit on the service sector. Again looking at the table at the top of the notebook, exports of financial and professional services (consulting/audit/accounting services in particular) make up 50% of the final demand for these sectors! It is well known that the UK, and London in particular provides extensive financial and professional services to the EU and the rest of the world. Unfortunately we do not have the detailed breakdown for EU vs non-EU exports, but a rule of thumb is that ~40% of UK service exports go to the UK. Let's then assume that a no-deal Brexit reduces the final demand for these sectors by 25%. What is the impact?

In [5]:
uk_IO_widget(shockSector_init = ['Financial and insurance', 'Professional and support activities'], 
             shockSign_init = 'Negative', shockSize_init = 25, NumRounds_init = 5, Normalise_init = False, 
                 plotType_init = 'Shocks over time')

In [6]:
uk_IO_widget(shockSector_init = ['Financial and insurance', 'Professional and support activities'], 
             shockSign_init = 'Negative', shockSize_init = 25, NumRounds_init = 5, Normalise_init = False, 
                 plotType_init = 'Cumulative shocks')

Here we can see an interesting contrast between services and production. The shock to production did not seem to ripple out much to other sectors, however the effect on production itself did grow over time. For a shock to the financial and professional services sectors, the initial shock in those sectors decay quicker than was the case for production, however the shock is transmitted to many more sectors including production. This probably because finance, insurance and professional services are much more connected to every sector in the economy.

It is also interesting to note that the overall effect on production is about the same size in both cases. The cumulative effect of the shock is to reduce total production by about £140 to £160 billion in both cases. Note, however, that the initial reduction of demand was much larger for the services sector in % terms. Finally if we combine both 'Brexit' shocks together we get a number, £300 billion or 6% of total production, which is actually quite close to the forecasts made by professional economists for a no-deal Brexit. **A word of caution**: this is very crude forecasting method, many strong assumptions are being made here under the hood (like production depending linearly on inputs), and we are ignoring many factors (imports, for example). But as an illustration it works, and that's my goal here.

#### In conclusion...

Hopefully you can see how matrix methods have allowed us to basically take what is an accounting table and turn it into a method of exploring the input and output connections of the sectors in the economy, and even figuring out how shocked individual sectors affect other sectors. All this simply from taking a standard table of data and applying some matrix magic... Feel free to play around with these examples, experiment and try to build your own horror stories (or miracles?) for the UK economy by shocking different sets of sectors.
